In [ ]:
import csv
import os

import numpy as np
import pandas as pd
#from owid_downloader import GenerateTrainingData
#from utils import date_today, gravity_law_commute_dist

os.environ['NUMEXPR_MAX_THREADS'] = '16'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

import pickle
import matplotlib.pyplot as plt
import dgl
import torch
from torch import nn
import torch.nn.functional as F
from model import STAN

import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
#GenerateTrainingData().download_jhu_data('2020-08-01', '2020-12-01')
start_date = '2021-01-01'
end_date = '2021-05-31'

In [ ]:
# data processing
raw_data = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv', usecols=["location", "date", "total_cases", "new_cases_smoothed", "total_deaths",
                    "new_deaths", "total_vaccinations", "people_fully_vaccinated", "new_vaccinations", "population"])
raw_data['date'] = pd.to_datetime(raw_data['date'])
mask = (raw_data['date'] >= start_date) & (raw_data['date'] <= end_date) # & (raw_data['location'].isin(countries))
raw_data = raw_data.loc[mask]
#print(raw_data[raw_data['location'] == 'United States']['total_cases'].values[0])
countries = []
loc_list = list(raw_data['location'].unique())
# only include countries that have more than 1000 total cases on start date and at least 1 death
for loc in loc_list:
    if raw_data[raw_data['location'] == loc]["total_cases"].values[0] > 1000 and \
        raw_data[raw_data['location'] == loc]["total_deaths"].values[0] > 0:
        countries.append(loc)
# hard-coded; these are problematic locations (non-countries) that need to be removed
countries.remove("European Union")
countries.remove("Europe")
countries.remove("Africa")
countries.remove("Asia")
countries.remove("North America")
countries.remove("Oceania")
countries.remove("South America")
countries.remove("World")
countries.remove("Tajikistan")
mask = raw_data['location'].isin(countries)
raw_data = raw_data.loc[mask]
print(len(raw_data['location'].unique()))

In [ ]:
# Generate Graph
# add flight neighbors
# for now, add a connection if there is any flight between the two countries between start and end date
loc_list = list(raw_data['location'].unique())
flight_counts = pd.read_csv('processed_flights/flight_counts_2021_all_to_05.csv')
adj_map = {}
for each_loc in loc_list:
    df = flight_counts.loc[flight_counts["origin_country"] == each_loc]
    adj_map[each_loc] = set(df["destination_country"].unique())
flight_counts['day'] = pd.to_datetime(flight_counts['day'])

In [ ]:
# add land neighbors
import csv
neighbor_reader = csv.reader(open('neighbors.csv', 'r'))
neighbors = {}
for row in neighbor_reader:
   neighbors[row[0]] = row[1].split(',')
for each_loc,connected in adj_map.items():
    for neighbor in neighbors[each_loc]:
        if neighbor in loc_list:
            connected.add(neighbor)

In [ ]:
# create graph
rows = []
cols = []
for each_loc in adj_map:
    for each_loc2 in adj_map[each_loc]:
        if each_loc in loc_list and each_loc2 in loc_list:
            rows.append(loc_list.index(each_loc))
            cols.append(loc_list.index(each_loc2))
#print(rows)
#print(cols)
g = dgl.graph((rows, cols))
print(g.number_of_nodes)

In [ ]:
# import networkx as nx
# nx_g = g.to_networkx()
# pos = nx.kamada_kawai_layout(nx_g)
# plt.figure(1,figsize=(8,8)) 
# nx.draw(nx_g, pos, with_labels=True, node_color=[[.7, .7, .7]])
# plt.show()

In [ ]:
#Preprocess features

#active_cases = []
confirmed_cases = []
new_cases = []
new_vaccinations = []
fully_vaccinated = []
death_cases = []
static_feat = []

for i, each_loc in enumerate(loc_list):
    confirmed_cases.append(raw_data[raw_data['location'] == each_loc]['total_cases'])
    new_cases.append(raw_data[raw_data['location'] == each_loc]['new_cases_smoothed'])
    new_vaccinations.append(raw_data[raw_data['location'] == each_loc]['new_vaccinations'])
    fully_vaccinated.append(raw_data[raw_data['location'] == each_loc]['people_fully_vaccinated'])
    death_cases.append(raw_data[raw_data['location'] == each_loc]['total_deaths'])
    static_feat.append(np.array(raw_data[raw_data['location'] == each_loc][['population']]))
confirmed_cases = np.nan_to_num(np.array(confirmed_cases))
death_cases = np.nan_to_num(np.array(death_cases)[:, 14:])
new_cases = np.nan_to_num(np.array(new_cases)[:, 14:])
new_vaccinations = np.nan_to_num(np.array(new_vaccinations)[:, 14:])
fully_vaccinated = np.nan_to_num(np.array(fully_vaccinated))
static_feat = np.nan_to_num(np.array(static_feat)[:, 0, :])

import copy
# active = confirmed(today) - confirmed(14 days ago)
cases_copy = copy.deepcopy(confirmed_cases)
active = []
for loc in confirmed_cases:
    active_loc = []
    for i in range(14, len(loc)):
        active_loc.append(loc[i] - loc[i-14])
    active.append(active_loc)
active_cases = np.array(active)

confirmed_cases = confirmed_cases[:, 14:]
fully_vaccinated = fully_vaccinated[:, 14:]

recovered_cases = confirmed_cases - active_cases - death_cases + 0.94*fully_vaccinated
susceptible_cases = np.expand_dims(static_feat[:, 0], -1) - active_cases - recovered_cases

# Batch_feat: new_cases(dI), dR, dS
#dI = np.array(new_cases)
dI = np.concatenate((np.zeros((active_cases.shape[0],1), dtype=np.float32), np.diff(active_cases)), axis=-1)
dR = np.concatenate((np.zeros((recovered_cases.shape[0],1), dtype=np.float32), np.diff(recovered_cases)), axis=-1)
dS = np.concatenate((np.zeros((susceptible_cases.shape[0],1), dtype=np.float32), np.diff(susceptible_cases)), axis=-1)
# number of new fully vaccinated each day
Vt = np.concatenate((np.zeros((fully_vaccinated.shape[0],1), dtype=np.float32), np.diff(fully_vaccinated)), axis=-1)
print("done")

In [ ]:
#Build normalizer
normalizer = {'S':{}, 'I':{}, 'R':{}, 'dS':{}, 'dI':{}, 'dR':{}, 'Vt':{}}

for i, each_loc in enumerate(loc_list):
    normalizer['S'][each_loc] = (np.mean(susceptible_cases[i]), np.std(susceptible_cases[i]))
    normalizer['I'][each_loc] = (np.mean(active_cases[i]), np.std(active_cases[i]))
    normalizer['R'][each_loc] = (np.mean(recovered_cases[i]), np.std(recovered_cases[i]))
    normalizer['dI'][each_loc] = (np.mean(dI[i]), np.std(dI[i]))
    normalizer['dR'][each_loc] = (np.mean(dR[i]), np.std(dR[i]))
    normalizer['dS'][each_loc] = (np.mean(dS[i]), np.std(dS[i]))
    normalizer['Vt'][each_loc] = (np.mean(Vt[i]), np.std(Vt[i]))

In [ ]:
def prepare_data(data, sum_I, sum_R, Vt, edges_df, start, history_window=8, pred_window=14, slide_step=7):
    # Data shape n_loc, timestep, n_feat
    # Reshape to n_loc, t, history_window*n_feat
    n_loc = data.shape[0]
    timestep = data.shape[1]
    n_feat = data.shape[2]
    
    x = []
    y_I = []
    y_R = []
    last_I = []
    last_R = []
    concat_I = []
    concat_R = []
    concat_Vt = []
    edges = []
    for i in range(0, timestep, slide_step):
        if i+history_window+pred_window-1 >= timestep or i+history_window >= timestep:
            break
        x.append(data[:, i:i+history_window, :].reshape((n_loc, history_window*n_feat)))
        
        concat_I.append(data[:, i+history_window-1, 0])
        concat_R.append(data[:, i+history_window-1, 1])
        last_I.append(sum_I[:, i+history_window-1])
        last_R.append(sum_R[:, i+history_window-1])

        y_I.append(data[:, i+history_window:i+history_window+pred_window, 0])
        y_R.append(data[:, i+history_window:i+history_window+pred_window, 1])

        concat_Vt.append(Vt[:, i+history_window:i+history_window+pred_window])
        
        e_matrix = np.zeros((n_loc, n_loc)) # edge weight matrix for every time period 
        df = edges_df.groupby(["origin_country"])
        for loc in range(n_loc):
            try:
                src_df = df.get_group(loc_list[loc])
                src_df = src_df.groupby(["destination_country"])
                for loc2 in range(n_loc):
                    try:
                        dst_df = src_df.get_group(loc_list[loc2])
                        dst_df = dst_df.loc[(dst_df['day'] >= (pd.to_datetime(start) + pd.DateOffset(days=i)))]
                        dst_df = dst_df.loc[(dst_df['day'] < (pd.to_datetime(start) + pd.DateOffset(days=i+history_window-1)))]
                        e_matrix[loc, loc2] = dst_df['flight_count'].sum()
                    except:
                        continue
            except:
                continue
        # normalize matrix (doubly stochastic, see https://arxiv.org/pdf/1809.02709.pdf)
        # step 1: row normalize
        norm = np.sum(e_matrix, axis=1, keepdims=True)
        norm[norm==0] = 1e-10
        norm = 1.0 / norm
        P = e_matrix * norm

        # step 2: P @ P^T / column_norm
        norm = np.sum(P, axis=0, keepdims=True)
        norm[norm==0] = 1e-10
        norm = 1.0 / norm

        PT = np.transpose(P, (1, 0))
        P = np.multiply(P, norm)
        T = np.matmul(P, PT)
        edges.append(T) # n_rows = # countries, n_cols = # countries
        
    
    x = np.array(x, dtype=np.float32).transpose((1, 0, 2))
    last_I = np.array(last_I, dtype=np.float32).transpose((1, 0))
    last_R = np.array(last_R, dtype=np.float32).transpose((1, 0))
    concat_I = np.array(concat_I, dtype=np.float32).transpose((1, 0))
    concat_R = np.array(concat_R, dtype=np.float32).transpose((1, 0))
    y_I = np.array(y_I, dtype=np.float32).transpose((1, 0, 2))
    y_R = np.array(y_R, dtype=np.float32).transpose((1, 0, 2))
    concat_Vt = np.array(concat_Vt, dtype=np.float32).transpose((1, 0, 2))
    return x, last_I, last_R, concat_I, concat_R, y_I, y_R, concat_Vt, edges

In [ ]:
def squish_edges(E):
    # use src and dst (rows and cols) to make E matrix
    edges = []
    for M in E:
        edges_flat = []
        for i in range(len(rows)):
            edges_flat.append(M[rows[i]][cols[i]])
        edges.append(edges_flat)
    return np.array(edges)

In [ ]:
valid_window = 29
test_window = 29

history_window=14 # days of information
pred_window=14 # predicts future # of days
slide_step=3 # increment

dynamic_feat = np.concatenate((np.expand_dims(dI, axis=-1), np.expand_dims(dR, axis=-1), np.expand_dims(dS, axis=-1)), axis=-1)
    
#Normalize
for i, each_loc in enumerate(loc_list):
    dynamic_feat[i, :, 0] = (dynamic_feat[i, :, 0] - normalizer['dI'][each_loc][0]) / normalizer['dI'][each_loc][1]
    dynamic_feat[i, :, 1] = (dynamic_feat[i, :, 1] - normalizer['dR'][each_loc][0]) / normalizer['dR'][each_loc][1]
    dynamic_feat[i, :, 2] = (dynamic_feat[i, :, 2] - normalizer['dS'][each_loc][0]) / normalizer['dS'][each_loc][1]
    # vaccinations don't need to be normalized
    #mean_vax = normalizer['Vt'][each_loc][0]
    #if mean_vax != 0:
    #   Vt[i] = (Vt[i] - mean_vax) / normalizer['Vt'][each_loc][1]
dI_mean = []
dI_std = []
dR_mean = []
dR_std = []

for i, each_loc in enumerate(loc_list):
    dI_mean.append(normalizer['dI'][each_loc][0])
    dR_mean.append(normalizer['dR'][each_loc][0])
    dI_std.append(normalizer['dI'][each_loc][1])
    dR_std.append(normalizer['dR'][each_loc][1])

dI_mean = np.array(dI_mean)
dI_std = np.array(dI_std)
dR_mean = np.array(dR_mean)
dR_std = np.array(dR_std)

#Split train-test
train_feat = dynamic_feat[:, :-valid_window-test_window, :]
val_feat = dynamic_feat[:, -valid_window-test_window:-test_window, :]
test_feat = dynamic_feat[:, -test_window:, :]

valid_start_date = pd.to_datetime(end_date) + pd.DateOffset(days=-valid_window) + pd.DateOffset(days=-test_window)
test_start_date = pd.to_datetime(end_date) + pd.DateOffset(days=-test_window)

train_edges = flight_counts[(flight_counts["day"] >= start_date) & (flight_counts["day"] < valid_start_date)]
val_edges = flight_counts[(flight_counts["day"] >= valid_start_date) & (flight_counts["day"] < test_start_date)]
test_edges = flight_counts[(flight_counts["day"] >= test_start_date) & (flight_counts["day"] < end_date)]

train_x, train_I, train_R, train_cI, train_cR, train_yI, train_yR, train_Vt, train_edges = prepare_data(train_feat, active_cases[:, :-valid_window-test_window], recovered_cases[:, :-valid_window-test_window], Vt[:, :-valid_window-test_window], train_edges, start_date, history_window, pred_window, slide_step)

val_x, val_I, val_R, val_cI, val_cR, val_yI, val_yR, val_Vt, val_edges = prepare_data(val_feat, active_cases[:, -valid_window-test_window:-test_window], recovered_cases[:, -valid_window-test_window:-test_window], Vt[:, -valid_window-test_window:-test_window], val_edges, valid_start_date, history_window, pred_window, slide_step)

test_x, test_I, test_R, test_cI, test_cR, test_yI, test_yR, test_Vt, test_edges = prepare_data(test_feat, active_cases[:, -test_window:], recovered_cases[:, -test_window:], Vt[:, -test_window:], test_edges, test_start_date, history_window, pred_window, slide_step)

train_edges = squish_edges(train_edges)
val_edges = squish_edges(val_edges)
test_edges = squish_edges(test_edges)

In [ ]:
print(train_edges.shape) # one edge array (len = # edges) for each timestep
print(train_x.shape) # one array of features for each timestep for each location
print(val_edges.shape)
print(val_x.shape)
print(test_edges.shape)
print(test_x.shape)

In [ ]:
#Build STAN model

in_dim = 3*history_window
hidden_dim1 = 32
hidden_dim2 = 32
gru_dim = 32
num_heads = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

g = g.to(device)
model = STAN(g, in_dim, hidden_dim1, hidden_dim2, gru_dim, num_heads, pred_window, device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss()

In [ ]:
model

In [ ]:
train_x = torch.tensor(train_x).to(device)
train_I = torch.tensor(train_I).to(device)
train_R = torch.tensor(train_R).to(device)
train_cI = torch.tensor(train_cI).to(device)
train_cR = torch.tensor(train_cR).to(device)
train_yI = torch.tensor(train_yI).to(device)
train_yR = torch.tensor(train_yR).to(device)
train_Vt = torch.tensor(train_Vt).to(device)
train_edges = torch.tensor(train_edges).to(device)

val_x = torch.tensor(val_x).to(device)
val_I = torch.tensor(val_I).to(device)
val_R = torch.tensor(val_R).to(device)
val_cI = torch.tensor(val_cI).to(device)
val_cR = torch.tensor(val_cR).to(device)
val_yI = torch.tensor(val_yI).to(device)
val_yR = torch.tensor(val_yR).to(device)
val_Vt = torch.tensor(val_Vt).to(device)
val_edges = torch.tensor(val_edges).to(device)

test_x = torch.tensor(test_x).to(device)
test_I = torch.tensor(test_I).to(device)
test_R = torch.tensor(test_R).to(device)
test_cI = torch.tensor(test_cI).to(device)
test_cR = torch.tensor(test_cR).to(device)
test_yI = torch.tensor(test_yI).to(device)
test_yR = torch.tensor(test_yR).to(device)
test_Vt = torch.tensor(test_Vt).to(device)
test_edges = torch.tensor(test_edges).to(device)

dI_mean = torch.tensor(dI_mean, dtype=torch.float32).to(device).reshape((dI_mean.shape[0], 1, 1))
dI_std = torch.tensor(dI_std, dtype=torch.float32).to(device).reshape((dI_mean.shape[0], 1, 1))
dR_mean = torch.tensor(dR_mean, dtype=torch.float32).to(device).reshape((dI_mean.shape[0], 1, 1))
dR_std = torch.tensor(dR_std, dtype=torch.float32).to(device).reshape((dI_mean.shape[0], 1, 1))

N = torch.tensor(static_feat[:, 0], dtype=torch.float32).to(device).unsqueeze(-1)

In [ ]:
def train_adaptive_loss(pred_I, pred_I_sir, pred_R, pred_R_sir, true_I, true_R, pred_window=14):
    total_loss = 0
    for timestep in range(len(pred_I)):
        for day in range(1, pred_window+1):
            sir_weight = day/(pred_window+1)
            pred_weight = 1-sir_weight
            sir_loss = sir_weight*((pred_I_sir[timestep][day-1] - true_I[timestep][day-1])**2) + sir_weight*((pred_R_sir[timestep][day-1] - true_R[timestep][day-1])**2)
            pred_loss = pred_weight*((pred_I[timestep][day-1] - true_I[timestep][day-1])**2) + pred_weight*((pred_R[timestep][day-1] - true_R[timestep][day-1])**2)
            total_loss += sir_loss + pred_loss
    return total_loss / pred_window

def val_adaptive_loss(pred_I, pred_I_sir, true_I, pred_window=14):
    total_loss = 0
    for day in range(1, pred_window+1):
        sir_weight = day/(pred_window+1)
        pred_weight = 1-sir_weight
        sir_loss = sir_weight*((pred_I_sir[day-1] - true_I[day-1])**2)
        pred_loss = pred_weight*((pred_I[day-1] - true_I[day-1])**2)
        total_loss += sir_loss + pred_loss
    return total_loss / pred_window

In [ ]:
#Train STAN
all_loss = []
file_name = './save/stan'
min_loss = 1e20

train_locs = ["United States", "United Kingdom"]

for epoch in range(50):
    global_train_loss = 0
    global_val_loss = 0
    for loc in train_locs:
        cur_loc = loc_list.index(loc)
        model.train()
        optimizer.zero_grad()
        
        active_pred, recovered_pred, phy_active, phy_recover, _ = model(train_x, train_cI[cur_loc], train_cR[cur_loc], N[cur_loc], train_I[cur_loc], train_R[cur_loc], V=train_Vt[cur_loc], e_weights=train_edges)
        phy_active = (phy_active - dI_mean[cur_loc]) / dI_std[cur_loc]
        phy_recover = (phy_recover - dR_mean[cur_loc]) / dR_std[cur_loc]
        # SIR loss = (day) / (pred_window + 1); dynamics loss = 1 - SIR loss 
        #loss = criterion(active_pred.squeeze(), train_yI[cur_loc].squeeze())+criterion(recovered_pred.squeeze(), train_yR[cur_loc].squeeze()) \
        #    + 0.1*criterion(phy_active.squeeze(), train_yI[cur_loc].squeeze())+0.1*criterion(phy_recover.squeeze(), train_yR[cur_loc].squeeze())
        loss = train_adaptive_loss(active_pred.squeeze(), phy_active.squeeze(), recovered_pred.squeeze(), phy_recover.squeeze(), train_yI[cur_loc].squeeze(), train_yR[cur_loc].squeeze(), pred_window)

        loss.backward()
        optimizer.step()
        all_loss.append(loss.item())
        
        model.eval()
        _, _, _, _, prev_h = model(train_x, train_cI[cur_loc], train_cR[cur_loc], N[cur_loc], train_I[cur_loc], train_R[cur_loc], V=train_Vt[cur_loc], e_weights=train_edges)
        val_active_pred, val_recovered_pred, val_phy_active, val_phy_recover, _ = model(val_x, val_cI[cur_loc], val_cR[cur_loc], N[cur_loc], val_I[cur_loc], val_R[cur_loc], prev_h, V=val_Vt[cur_loc], e_weights=val_edges)
        
        val_phy_active = (val_phy_active - dI_mean[cur_loc]) / dI_std[cur_loc]
        # SIR loss = (day) / (pred_window + 1); dynamics loss = 1 - SIR loss 
        # change loss here 
        #val_loss = criterion(val_active_pred.squeeze(), val_yI[cur_loc].squeeze()) + 0.1*criterion(val_phy_active.squeeze(), val_yI[cur_loc].squeeze())
        val_loss = val_adaptive_loss(val_active_pred.squeeze(), val_phy_active.squeeze(), val_yI[cur_loc].squeeze(), pred_window)
        global_train_loss += loss
        global_val_loss += val_loss
    if (global_val_loss + global_train_loss) / 2 < min_loss:   
        state = {
            'state': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        torch.save(state, file_name)
        min_loss = (global_val_loss + global_train_loss) / 2
        print('-----Save best model-----')
    print('Epoch %d, Loss %.2f, Val loss %.2f'%(epoch, global_train_loss.item(), global_val_loss.item()))

In [ ]:
#Pred with STAN
file_name = './save/stan'
checkpoint = torch.load(file_name)
model.load_state_dict(checkpoint['state'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

prev_x = torch.cat((train_x, val_x), dim=1)
prev_I = torch.cat((train_I, val_I), dim=1)
prev_R = torch.cat((train_R, val_R), dim=1)
prev_cI = torch.cat((train_cI, val_cI), dim=1)
prev_cR = torch.cat((train_cR, val_cR), dim=1)
prev_Vt = torch.cat((train_Vt, val_Vt), dim=1)
prev_edges = torch.cat((train_edges, val_edges), dim=0)

train_locs.reverse()

for loc_name in train_locs:
    cur_loc = loc_list.index(loc_name)
    prev_active_pred, _, prev_phy_active_pred, _, h = model(prev_x, prev_cI[cur_loc], prev_cR[cur_loc], N[cur_loc], prev_I[cur_loc], prev_R[cur_loc], V=prev_Vt[cur_loc], e_weights=prev_edges)

    test_pred_active, test_pred_recovered, test_pred_phy_active, test_pred_phy_recover, _ = model(test_x, test_cI[cur_loc], test_cR[cur_loc], N[cur_loc], test_I[cur_loc], test_R[cur_loc], h, V=test_Vt[cur_loc], e_weights=test_edges)

    print('Estimated beta in SIR model is %.2f'%model.alpha_scaled)
    print('Estimated gamma in SIR model is %.2f'%model.beta_scaled)
    print('Estimated theta in SIR model is %.2f'%model.theta_scaled)

    #Cumulate predicted dI
    pred_I = []
    sir_I = []

    for i in range(test_pred_active.size(1)):
        # below is regular prediction
        cur_pred = (test_pred_active[0, i, :].detach().cpu().numpy() * dI_std[cur_loc].reshape(1, 1).detach().cpu().numpy()) + dI_mean[cur_loc].reshape(1, 1).detach().cpu().numpy()
        # below is SIR model prediction
        sir_pred = test_pred_phy_active[0, i, :].detach().cpu().numpy()
        # below is average of the two predictions
        #cur_pred = (cur_pred + test_pred_phy_active[0, i, :].detach().cpu().numpy()) / 2
        cur_pred = np.cumsum(cur_pred)
        cur_pred = cur_pred + test_I[cur_loc, i].detach().cpu().item()
        pred_I.append(cur_pred)

        sir_pred = np.cumsum(sir_pred)
        sir_pred = sir_pred + test_I[cur_loc, i].detach().cpu().item()
        sir_I.append(sir_pred)
    pred_I = np.array(pred_I)
    sir_I = np.array(sir_I)

    #I_true = get_real_y(active_cases[:], history_window, pred_window, slide_step)
    I_true = active_cases[cur_loc][-test_window:]
    I_true = I_true[history_window:history_window+pred_window]


    import matplotlib.dates as mdates
    dates = [test_start_date + pd.DateOffset(days=i+history_window) for i in range(pred_window)]
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
    #plt.plot(I_true[cur_loc, -1, :],c='r', label='Ground truth')
    stan_res = pickle.load(open(f"stan_results/{loc_name}_stan_results_new", "rb"))
    stan_sirv_res = pickle.load(open(f"stan_results/{loc_name}_stan_results_sirv", "rb"))
    plt.plot(dates, I_true, c='r', label='Ground truth')
    plt.plot(dates, stan_res[-1, :], c='g', label="STAN (SIRV)")
    #plt.plot(dates, sir_I[-1, :], c='purple', label="SIRVIC")
    plt.plot(dates, pred_I[-1, :],c='b', label='Proposed')
    plt.gcf().autofmt_xdate()
    plt.legend()
    plt.ylabel("Number of Active Cases")
    plt.title(loc_name)
    plt.show()

    prop_mse = ((pred_I[0] - I_true)**2).mean()
    sirvic_mse = ((sir_I[0] - I_true)**2).mean()
    stan_mse = ((stan_res[0] - I_true)**2).mean()
    print("MSE of Models")
    print(f"Proposed: {prop_mse}")
    print(f"STAN: {stan_mse}")
    print(f"SIRVIC: {sirvic_mse}")

In [ ]:

# prop_mse = ((pred_I[0] - I_true)**2).mean()
# sirvic_mse = ((sir_I[0] - I_true)**2).mean()
# stan_mse = ((stan_res[0] - I_true)**2).mean()
# print("MSE of Models")
# print(f"Proposed: {prop_mse}")
# print(f"STAN: {stan_mse}")
# print(f"SIRVIC: {sirvic_mse}")

In [ ]:
# final graph visualization
import networkx as nx
import seaborn as sns

def plot(g, attention, ax, nodes_to_plot=None, nodes_labels=None,
         edges_to_plot=None, nodes_pos=None, nodes_colors=None,
         edge_colormap=plt.cm.viridis, labels=[], edge_layer_2=None):
    """
    Visualize edge attentions by coloring edges on the graph.
    g: nx.DiGraph
        Directed networkx graph
    attention: list
        Attention values corresponding to the order of sorted(g.edges())
    ax: matplotlib.axes._subplots.AxesSubplot
        ax to be used for plot
    nodes_to_plot: list
        List of node ids specifying which nodes to plot. Default to
        be None. If None, all nodes will be plot.
    nodes_labels: list, numpy.array
        nodes_labels[i] specifies the label of the ith node, which will
        decide the node color on the plot. Default to be None. If None,
        all nodes will have the same canonical label. The nodes_labels
        should contain labels for all nodes to be plot.
    edges_to_plot: list of 2-tuples (i, j)
        List of edges represented as (source, destination). Default to
        be None. If None, all edges will be plot.
    nodes_pos: dictionary mapping int to numpy.array of size 2
        Default to be None. Specifies the layout of nodes on the plot.
    nodes_colors: list
        Specifies node color for each node class. Its length should be
        bigger than number of node classes in nodes_labels.
    edge_colormap: plt.cm
        Specifies the colormap to be used for coloring edges.
    """
    if nodes_to_plot is None:
        nodes_to_plot = sorted(g.nodes())
    if edges_to_plot is None:
        edges_to_plot = sorted(g.edges())
    nx.draw_networkx_edges(g, nodes_pos, edgelist=edges_to_plot,
                           edge_color=attention, edge_cmap=edge_colormap,
                           width=2, alpha=0.5, ax=ax, edge_vmin=0,
                           edge_vmax=1)
    if edge_layer_2 is not None:
        nx.draw_networkx_edges(g, nodes_pos, edgelist=edge_layer_2[0],
                           edge_color=edge_layer_2[1], edge_cmap=edge_colormap,
                           width=2, alpha=0.5, ax=ax, edge_vmin=0,
                           edge_vmax=1)

    if nodes_colors is None:
        nodes_colors = sns.color_palette("deep", max(nodes_labels) + 1)

    nx.draw_networkx_nodes(g, nodes_pos, nodelist=nodes_to_plot, ax=ax, node_size=20,
                           node_color=[nodes_colors[nodes_labels[v - 1]] for v in nodes_to_plot],
                            alpha=0.9)
    loc_labels = {}
    for node in g.nodes():
        if node in labels:
            loc_labels[node] = loc_list[node]
    nx.draw_networkx_labels(g, nodes_pos, loc_labels, font_size=16)

nx_g = model.g.cpu().to_networkx(edge_attrs=['e'])
#print(nx_g.nodes())
attention = []
mid_attention = []
mid_edges = []
for e in sorted(nx_g.edges()):
    a = nx_g.get_edge_data(e[0], e[1])[0]['e'].item()
    attention.append(a)
    if a > 0.1 and a < 0.9:
        mid_attention.append(a)
        mid_edges.append(e)
labels = [0 if i not in train_locs else 1 for i in range(len(nx_g.nodes()))]
loc_i_list = [loc_list.index(loc) for loc in train_locs]
pos = nx.spring_layout(nx_g, k=0.25, iterations=20)  # positions for all nodes

fig, ax = plt.subplots(figsize=(16, 12))
plot(nx_g, attention, ax, nodes_pos=pos, nodes_labels=labels, nodes_colors=["red" if i not in train_locs else "blue" for i in range(len(nx_g.nodes()))], edge_layer_2=(mid_edges, mid_attention))
ax.set_axis_off()
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=0, vmax=1))
sm.set_array([])
plt.colorbar(sm, fraction=0.046, pad=0.01)
plt.show()